In [2]:
import pyprob
from pyprob.distributions import Normal
import pyprob.diagnostics
import torch
import pandas
import os

def parse_n_infect_from_output(file_name='/home/examples/output.txt'):
    data = pandas.read_csv(file_name, sep='\t', names = ["survey_number", "third_dimension", "measure", "value"])
    n_infect = torch.from_numpy(data['value'][:12].values)
    return n_infect

PATH = '/home/examples/test/'
def forward_before():
    exists = os.path.isfile(PATH + 'output.txt')
    if exists:
        print('Removing output...')
        os.remove(PATH+'output.txt')
    
def forward_after():
    n_infect = parse_n_infect_from_output()
    likelihood = Normal(n_infect, 0.01)
    pyprob.observe(likelihood, name='n_infect')

# This will save in /code/notebooks
file_name = 'trace'
address_dict_file_name = 'address_dict'
use_address_base=True
model = pyprob.RemoteModel('ipc://@openmalaria_probprog', before_forward_func=forward_before ,after_forward_func=forward_after, address_dict_file_name=address_dict_file_name)

## Generating the Prior

In [3]:
prior = model.prior_traces(num_traces=3, file_name=file_name)

Time spent  | Time remain.| Progress             | Trace | Traces/sec
ppx (Python): zmq.REQ socket connecting to server ipc://@openmalaria_probprog
ppx (Python): This system        : pyprob 0.13.2.dev18
ppx (Python): Connected to system: pyprob_cpp 0.1.9 (master:7290be2)
ppx (Python): Model name         : OpenMalaria probprog
Removing output...0:14:09 | #################### | 2/2 | 0.00       



## Analysing the Prior

In [ ]:
# print(len(prior))
# trace=prior._values[0]
prior._values
trace=prior._values[0]
var = trace.variables
# len(var)
# print(var)
count  = 1
for k in trace.variables_dict_address_base:
    print(" Address {2} : {0} \n Value : {1} \n\n".format(k,trace.variables_dict_address_base[k],count))
    count += 1
pyprob.diagnostics.graph(prior, file_name="mpenny_10")
print(prior.save_metadata)

## Generating to the posterior by conditioning on the number of infected individuals for each demographic

In [ ]:
 posterior_dist = model.posterior_traces(num_traces=2, inference_engine=pyprob.InferenceEngine.RANDOM_WALK_METROPOLIS_HASTINGS, observe={'n_infect': parse_n_infect_from_output})

Removing output...


#### View observations

In [ ]:
# ground_truth_trace = next(model._trace_generator())
prior.named_variables['n_infect'].value

In [ ]:
is_posterior_traces = model.posterior_traces(observe={'n_infect': ground_truth_prevalence}, num_traces=5)
is_posterior_population = is_posterior_traces.map(lambda trace: trace.named_variables['population'].value)
is_posterior_prevalence = is_posterior_population.map(get_prevalence)

## Plot Histogram of Addresses

In [ ]:
# we currently do not have a ground truth for this scenario run with a small pop size
pyprob.diagnostics.address_histograms(dists, plot=True, plot_show=True, file_name=file_name, ground_truth_trace=ground_truth_trace, use_address_base=use_address_base)

 Address 1 : [forward()+0x1f3; OM::Simulator::start(scnXml::Monitoring const&)+0x28a; OM::Population::createInitialHumans()+0x94; OM::Population::newHuman(OM::SimTime)+0x5c; OM::Host::Human::Human(OM::SimTime)+0x12b; OM::WithinHost::WHInterface::createWithinHostModel(double)+0x72; OM::WithinHost::CommonWithinHost::CommonWithinHost(double)+0x3a; OM::WithinHost::WHFalciparum::WHFalciparum(double)+0xe6; OM::util::random::gauss(double, double)+0xb4]__Normal 
 Value : Variable(name:None, control:True, replace:False, observable:False, observed:False, tagged:False, address:[forward()+0x1f3; OM::Simulator::start(scnXml::Monitoring const&)+0x28a; OM::Population::createInitialHumans()+0x94; OM::Population::newHuman(OM::SimTime)+0x5c; OM::Host::Human::Human(OM::SimTime)+0x12b; OM::WithinHost::WHInterface::createWithinHostModel(double)+0x72; OM::WithinHost::CommonWithinHost::CommonWithinHost(double)+0x3a; OM::WithinHost::WHFalciparum::WHFalciparum(double)+0xe6; OM::util::random::gauss(double, doub

'/Users/bradley/Documents/Projects/OpenMalaria/code/notebooks'